# Deploying GPT-2 to ODSP

In [ ]:
from transformers import pipeline
import requests
import mlflow

In [ ]:
# user and default API key, if the default environment file is used (NOT RECOMMENDED FOR ANYTHING OTHER THAN TESTING PURPOSES)

user = 'odsp'
key = 'odsp-odsp'

In [ ]:
model_id = 'gpt2'
model = pipeline('text-generation', model_id)

In [ ]:
data = ['How are you today?']
signature_output = mlflow.transformers.generate_signature_output(model, data)
signature = mlflow.models.infer_signature(data, signature_output)
model_config = {
    'max_new_tokens' : 50,
    'do_sample' : False,
    'return_full_text' : False
}

In [ ]:
mlflow.set_experiment('GPT2Tutorial')

with mlflow.start_run() as run:
    model_info = mlflow.transformers.log_model(
        model,
        'gpt2',
        model_config = model_config,
        signature = signature,
        registered_model_name = 'gpt2'
    )

In [ ]:
import requests

In [ ]:
with requests.Session() as sess:
    resp = sess.post('http://model-server:4488/models/predict/gpt2/pyfunc/1', json = {'data' : ['Tell me about yourself.']}, auth = ('odsp', 'odsp-odsp'))

In [ ]:
print(resp.json()['prediction'][0])